### Debug using requests

In [ ]:
import requests

In [ ]:
response = requests.post("http://localhost:8000/getInformation", json={"hi": "welcome to fastapi"})
print(response)

In [ ]:
response.json()

### Debug using async httpx

In [ ]:
import httpx
import time
start = time.time()
for i in range(20):

    async with httpx.AsyncClient() as client:
        r = await client.post('http://localhost:8000/getInformation', json={"hi": "welcome to fastapi"})
        print(r.json()['status'] == 'SUCCESS')
print(time.time() - start)


### Use throttle

In [ ]:
from aiolimiter import AsyncLimiter
import asyncio, time
import httpx


# class aiolimiter.AsyncLimiter(max_rate, time_period=60)
limiter = AsyncLimiter(5, .1) # 20 requests max would be acquired in .1 sec



start = time.time()
for i in range(20):
    async with limiter:
        async with httpx.AsyncClient() as client:
            print('sending:: ', time.time())
            r = await client.post('http://localhost:8000/getInformation', json={"hi": "welcome to fastapi"})
            print(r.json()['status'] == 'SUCCESS', 'received:', time.time())
print(time.time() - start)

In [ ]:
import asyncio
import time
from typing import Any

from httpx import AsyncClient, Limits

limits = httpx.Limits(max_connections=5)
client = httpx.AsyncClient(limits=limits)

REQUESTS = 10
URL = 'http://localhost:8000/getInformation'
TIMEOUT = 1


async def request_api(client: AsyncClient, i: int):
    t1 = time.process_time()
    print(f"===> {i}")
    r = await client.get(f"{URL}", timeout=TIMEOUT)

    return f"<=== {i} ({time.process_time() - t1:.3f}s)"


async def main():
    async with AsyncClient(limits=Limits(max_connections=5)) as client:
        aws = [asyncio.create_task(request_api(client, i)) for i in range(REQUESTS)]

        for coro in asyncio.as_completed(aws):
            earliest_result = await coro
            print(earliest_result)


if __name__ == "__main__":
    asyncio.run(main())